In [11]:
import xarray as xr
import pystac
import ast
import datetime
import json
import numpy as np
from pyproj import CRS

from pystac.extensions.datacube import (
    DatacubeExtension,
    Dimension,
    Variable,
)

# Spatiotempotal Asset Catalog (STAC)

### What are the STAC components?

The organizational hierarchy goes `Catalog` > `Collection` > `Item` > `Asset`.

However, the hierarchy is flexible in that some levels can be omitted. For example, the `Collection` is not needed in order to have `Items` or `Assets` - we could just have a `Catalog` full of `Items` and their associated `Assets`. But there are additional metadata fields available and improved search/browse capabilities when using `Collections`, so we will include them here for demonstration purposes.

## `Catalog`

In [84]:
?pystac.Catalog

Init signature:
pystac.Catalog(
    id: 'str',
    description: 'str',
    title: 'str | None' = None,
    stac_extensions: 'list[str] | None' = None,
    extra_fields: 'dict[str, Any] | None' = None,
    href: 'str | None' = None,
    catalog_type: 'CatalogType' = 'ABSOLUTE_PUBLISHED',
    strategy: 'HrefLayoutStrategy | None' = None,
)
Docstring:     
A PySTAC Catalog represents a STAC catalog in memory.

A Catalog is a :class:`~pystac.STACObject` that may contain children,
which are instances of :class:`~pystac.Catalog` or :class:`~pystac.Collection`,
as well as :class:`~pystac.Item` s.

Args:
    id : Identifier for the catalog. Must be unique within the STAC.
    description : Detailed multi-line description to fully explain the catalog.
        `CommonMark 0.29 syntax <https://commonmark.org/>`_ MAY be used for rich
        text representation.
    title : Optional short descriptive one-line title for the catalog.
    stac_extensions : Optional list of extensions the Catalog impl

## `Collection`

In [85]:
?pystac.Collection

Init signature:
pystac.Collection(
    id: 'str',
    description: 'str',
    extent: 'Extent',
    title: 'str | None' = None,
    stac_extensions: 'list[str] | None' = None,
    href: 'str | None' = None,
    extra_fields: 'dict[str, Any] | None' = None,
    catalog_type: 'CatalogType | None' = None,
    license: 'str' = 'other',
    keywords: 'list[str] | None' = None,
    providers: 'list[Provider] | None' = None,
    summaries: 'Summaries | None' = None,
    assets: 'dict[str, Asset] | None' = None,
    strategy: 'HrefLayoutStrategy | None' = None,
)
Docstring:     
A Collection extends the Catalog spec with additional metadata that helps
enable discovery.

Args:
    id : Identifier for the collection. Must be unique within the STAC.
    description : Detailed multi-line description to fully explain the
        collection. `CommonMark 0.29 syntax <https://commonmark.org/>`_ MAY
        be used for rich text representation.
    extent : Spatial and temporal extents that describe th

## `Item`

In [87]:
?pystac.Item

Init signature:
pystac.Item(
    id: 'str',
    geometry: 'dict[str, Any] | None',
    bbox: 'list[float] | None',
    datetime: 'Datetime | None',
    properties: 'dict[str, Any]',
    start_datetime: 'Datetime | None' = None,
    end_datetime: 'Datetime | None' = None,
    stac_extensions: 'list[str] | None' = None,
    href: 'str | None' = None,
    collection: 'str | Collection | None' = None,
    extra_fields: 'dict[str, Any] | None' = None,
    assets: 'dict[str, Asset] | None' = None,
)
Docstring:     
An Item is the core granular entity in a STAC, containing the core metadata
that enables any client to search or crawl online catalogs of spatial 'assets' -
satellite imagery, derived data, DEM's, etc.

Args:
    id : Provider identifier. Must be unique within the STAC.
    geometry : Defines the full footprint of the asset represented by this
        item, formatted according to
        `RFC 7946, section 3.1 (GeoJSON) <https://tools.ietf.org/html/rfc7946>`_.
    bbox :  Bounding

## `Asset`

In [111]:
?pystac.Asset

Init signature:
pystac.Asset(
    href: 'str',
    title: 'str | None' = None,
    description: 'str | None' = None,
    media_type: 'str | None' = None,
    roles: 'list[str] | None' = None,
    extra_fields: 'dict[str, Any] | None' = None,
) -> 'None'
Docstring:     
An object that contains a link to data associated with an Item or Collection that
can be downloaded or streamed.

Args:
    href : Link to the asset object. Relative and absolute links are both
        allowed.
    title : Optional displayed title for clients and users.
    description : A description of the Asset providing additional details,
        such as how it was processed or created. CommonMark 0.29 syntax MAY be used
        for rich text representation.
    media_type : Optional description of the media type. Registered Media Types
        are preferred. See :class:`~pystac.MediaType` for common media types.
    roles : Optional, Semantic roles (i.e. thumbnail, overview,
        data, metadata) of the asset.
  

# Create a test STAC
This STAC is just fake data so we can examine the structure. The fields populated below are the minimum required fields for the component dictionaries.

In [146]:
# the catalog
catalog = pystac.Catalog(id="test-catalog", description="Test catalog.")

# the collection
collection = pystac.Collection(
    id="test-collection",
    description="Test collection.",
    extent=pystac.Extent(
        spatial=pystac.SpatialExtent([[-180.0, -90.0, 180.0, 90.0]]),
        temporal=pystac.TemporalExtent([[None, None]]),
    ),
)

# the item
item = pystac.Item(
    id="test-item",
    geometry=None,
    bbox=[-10.0, -10.0, 10.0, 10.0],
    datetime=datetime.datetime.now(),
    properties={},
)

# the asset (file or link to file)
asset = pystac.Asset(
    href="https://example.com/data.nc",
    media_type=pystac.MediaType.NETCDF,
    roles=["data"],
    title="Test NetCDF Data",
)

item.add_asset("data", asset)
collection.add_item(item)
catalog.add_child(collection)

<Link rel=child target=<Collection id=test-collection>>

## Add Datacube Extension

This extension allows us to add more metadata to `Collections`, `Items`, and `Assets` - basically, we can describe multiple dimensions instead of just space and time, and multiple variables as well. Do we need to add this metadata to each component? Hard to say at this point - it's probably redundant, but there may be search / browse advantages to the metadata placement here that need further investigation.

The dimensions and variables are described in `Dimension` and `Variable` objects, respectively. See https://github.com/stac-extensions/datacube for more info.

In [147]:
# Define some dimension objects and a variable object

# time

time = Dimension(
    {
        "type": "temporal",
        "extent": ["2020-01-01T00:00:00Z", "2020-12-31T23:59:59Z"],
        "units": "day",
    }
)

# lat & lon

lat = Dimension(
    {
        "type": "spatial",
        "axis": "y",
        "extent": [50.0, 90.0],
        "reference_system": "EPSG:4326",
        "units": "degrees_north",
    }
)

lon = Dimension(
    {
        "type": "spatial",
        "axis": "x",
        "extent": [-180.0, 180.0],
        "reference_system": "EPSG:4326",
        "units": "degrees_east",
    }
)

# custom type (for model and scenario)

model = Dimension({"type": "model", "values": []})  # populate later with actual values

scenario = Dimension(
    {"type": "scenario", "values": []}
)  # populate later with actual values


# variable

tas = Variable(
    {
        "dimensions": ["time", "lat", "lon", "model", "scenario"],
        "type": "data",
        "nodata": "nan",
        "data_type": "float32",
        "description": "Surface temperature",
        "unit": "K",
    }
)

In [153]:
# collect the dimension and variable info in dicts so we can apply it to multiple STAC components
dim_dict = {
    "time": time,
    "lat": lat,
    "lon": lon,
    "model": model,
    "scenario": scenario,
}
var_dict = {"tas": tas}

# apply it to the collection, item, and asset
for comp in [collection, item, asset]:
    dc = DatacubeExtension.ext(comp, add_if_missing=True)
    dc.dimensions = dim_dict
    dc.variables = var_dict

## View the test STAC

In [155]:
# describe the catalog to see the basic structure
catalog.describe()

* <Catalog id=test-catalog>
    * <Collection id=test-collection>
      * <Item id=test-item>


In [158]:
# check out the catalog, collection, items, and assets in more detail by printing the JSON
# note that the parent/child relationship between these objects is not yet defined in the JSON
# currently these are only linked in memory in this script
# we will add the parent/child relationships using links when we save the catalog to disk in the next step
print(json.dumps(catalog.to_dict(), indent=4))

{
    "type": "Catalog",
    "id": "test-catalog",
    "stac_version": "1.1.0",
    "description": "Test catalog.",
    "links": [
        {
            "rel": "child",
            "href": null,
            "type": "application/json"
        }
    ]
}


In [159]:
print(json.dumps(collection.to_dict(), indent=4))

{
    "type": "Collection",
    "id": "test-collection",
    "stac_version": "1.1.0",
    "description": "Test collection.",
    "links": [
        {
            "rel": "item",
            "href": null,
            "type": "application/geo+json"
        },
        {
            "rel": "parent",
            "href": null,
            "type": "application/json"
        }
    ],
    "stac_extensions": [
        "https://stac-extensions.github.io/datacube/v2.2.0/schema.json"
    ],
    "cube:dimensions": {
        "time": {
            "type": "temporal",
            "extent": [
                "2020-01-01T00:00:00Z",
                "2020-12-31T23:59:59Z"
            ],
            "units": "day"
        },
        "lat": {
            "type": "spatial",
            "axis": "y",
            "extent": [
                50.0,
                90.0
            ],
            "reference_system": "EPSG:4326",
            "units": "degrees_north"
        },
        "lon": {
            "type": "s

In [160]:
print(json.dumps(item.to_dict(), indent=4))

{
    "type": "Feature",
    "stac_version": "1.1.0",
    "stac_extensions": [
        "https://stac-extensions.github.io/datacube/v2.2.0/schema.json"
    ],
    "id": "test-item",
    "geometry": null,
    "properties": {
        "cube:dimensions": {
            "time": {
                "type": "temporal",
                "extent": [
                    "2020-01-01T00:00:00Z",
                    "2020-12-31T23:59:59Z"
                ],
                "units": "day"
            },
            "lat": {
                "type": "spatial",
                "axis": "y",
                "extent": [
                    50.0,
                    90.0
                ],
                "reference_system": "EPSG:4326",
                "units": "degrees_north"
            },
            "lon": {
                "type": "spatial",
                "axis": "x",
                "extent": [
                    -180.0,
                    180.0
                ],
                "reference_system": 

In [161]:
print(json.dumps(asset.to_dict(), indent=4))

{
    "href": "https://example.com/data.nc",
    "type": "application/netcdf",
    "title": "Test NetCDF Data",
    "cube:dimensions": {
        "time": {
            "type": "temporal",
            "extent": [
                "2020-01-01T00:00:00Z",
                "2020-12-31T23:59:59Z"
            ],
            "units": "day"
        },
        "lat": {
            "type": "spatial",
            "axis": "y",
            "extent": [
                50.0,
                90.0
            ],
            "reference_system": "EPSG:4326",
            "units": "degrees_north"
        },
        "lon": {
            "type": "spatial",
            "axis": "x",
            "extent": [
                -180.0,
                180.0
            ],
            "reference_system": "EPSG:4326",
            "units": "degrees_east"
        },
        "model": {
            "type": "model",
            "values": []
        },
        "scenario": {
            "type": "scenario",
            "values":

## Add links and save

We need to normalize a directory that will act as the root directory for the `Catalog`'s JSON files. (See here for best practices: https://github.com/radiantearth/stac-spec/blob/v0.8.1/best-practices.md#catalog-layout)

We then save the `Catalog`, which will populate the parent/child links in all the STAC components.

In [162]:
dir = "./stac"
catalog.normalize_hrefs(dir)
catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)  # relative paths
# catalog.save(catalog_type=pystac.CatalogType.ABSOLUTE_PUBLISHED) # absolute paths

In [11]:
# check out the actual directory structure of the catalog using tree
!tree ./stac

./stac
├── catalog.json
└── test-collection
    ├── collection.json
    └── test-item
        └── test-item.json

3 directories, 3 files


In [ ]:
# recommend restarting the kernel at this point to clear memory / avoid local variable confusion
# then just re-import the necessary libraries and start at the cell below

# Create a STAC from real data

These are small coverage subsets. We want to use their actual dataset properties (dimensions, variable names, etc) to build a STAC `Catalog`.

In [21]:
# a small subset of the CF-compliant cmip6 monthly and cmip6 indicators datacubes in netcdf format
# this could also be a zarr store
fps = [
    "/Users/joshpaul/Desktop/cmip6_monthly_cf_subset.nc",
    "/Users/joshpaul/Desktop/cmip6_indicators_cf_subset.nc",
]

datasets = []

for fp in fps:
    ds = xr.open_dataset(fp, decode_cf=True)

    # replace model and scenario integers with actual names derived from the dimension's encoding dictionary
    # read the strings as dicts
    model_encodings = ast.literal_eval(ds.model.attrs["encoding"])
    scenario_encodings = ast.literal_eval(ds.scenario.attrs["encoding"])

    ds = ds.assign_coords(
        model=[model_encodings[int(m)] for m in ds.model.values],
        scenario=[scenario_encodings[int(s)] for s in ds.scenario.values],
    )

    # drop the crs variable (this is duplicate information - the spatial ref is already present)
    # ds = ds.drop_vars("crs")

    datasets.append(ds)

datasets

[<xarray.Dataset> Size: 27MB
 Dimensions:      (model: 3, scenario: 3, time: 1812, lat: 12, lon: 17)
 Coordinates:
   * time         (time) datetime64[ns] 14kB 1950-01-15 1950-02-15 ... 2100-12-15
   * lat          (lat) float64 96B 70.21 69.27 68.32 67.38 ... 61.73 60.79 59.84
   * lon          (lon) float64 136B -150.0 -148.8 -147.5 ... -131.2 -130.0
   * model        (model) <U13 156B 'CESM2' 'CNRM-CM6-1-HR' 'E3SM-2-0'
   * scenario     (scenario) <U10 120B 'historical' 'ssp126' 'ssp245'
 Data variables:
     crs          |S1 1B ...
     pr           (model, scenario, time, lat, lon) float32 13MB ...
     tas          (model, scenario, time, lat, lon) float32 13MB ...
     spatial_ref  int32 4B ...
 Attributes:
     Conventions:  CF-1.8
     contact:      uaf-snap-data-tools@alaska.edu
     description:  Monthly data from 13 CMIP6 models on a common grid, includi...
     institution:  Scenarios Network for Alaska and Arctic Planning, Universit...
     source:       CMIP6 model outpu

## Create a catalog with CMIP6 collections

Create a new `Catalog`, with one `Collection` / `Item` / `Asset` for each dataset. Use the actual dataset properties to fill in the metadata.? 

### `Catalog`

In [22]:
cmip6_catalog = pystac.Catalog(
    id="cmip6-catalog",
    description="Products derived from CMIP6 data.",
    title="CMIP6 Catalog",
)

### Get dataset info

This will be applied to each `Collection`, `Item`, and `Asset`

In [23]:
dataset_info_dict = {}

for dataset in datasets:

    # use dataset title as the dict key
    dataset_info_dict[dataset.attrs["title"]] = {}

    # basic info
    ds_id = dataset.attrs["title"].lower().replace(" ", "-")
    ds_desc = dataset.attrs["description"]
    ds_bbox = [
        float(dataset.lon.min()),
        float(dataset.lat.min()),
        float(dataset.lon.max()),
        float(dataset.lat.max()),
    ]
    # set these as datetime objects
    ds_startdate = dataset.time.values.min().astype("M8[ms]").astype("O")
    ds_enddate = dataset.time.values.max().astype("M8[ms]").astype("O")

    crs = CRS.from_wkt(dataset.crs.attrs["crs_wkt"])
    epsg_code = crs.to_epsg()
    ds_crs = str("EPSG:" + str(epsg_code))

    # add to dict
    dataset_info_dict[dataset.attrs["title"]]["id"] = ds_id
    dataset_info_dict[dataset.attrs["title"]]["description"] = ds_desc
    dataset_info_dict[dataset.attrs["title"]]["bbox"] = ds_bbox
    dataset_info_dict[dataset.attrs["title"]]["startdate"] = ds_startdate
    dataset_info_dict[dataset.attrs["title"]]["enddate"] = ds_enddate
    dataset_info_dict[dataset.attrs["title"]]["crs"] = ds_crs

    # define the dimensions
    time_dim = Dimension(
        {
            "type": "temporal",
            "extent": [
                ds_startdate.isoformat(),
                ds_enddate.isoformat(),
            ],
        }
    )

    lat_dim = Dimension(
        {
            "type": "spatial",
            "axis": "y",
            "extent": [float(dataset.lat.min()), float(dataset.lat.max())],
            "reference_system": ds_crs,
            "units": dataset.lat.attrs["units"],
        }
    )

    lon_dim = Dimension(
        {
            "type": "spatial",
            "axis": "x",
            "extent": [float(dataset.lon.min()), float(dataset.lon.max())],
            "reference_system": ds_crs,
            "units": dataset.lon.attrs["units"],
        }
    )

    model_dim = Dimension(
        {
            "type": "model",
            "values": [str(m) for m in dataset.model.values],
        }
    )

    scenario_dim = Dimension(
        {
            "type": "scenario",
            "values": [str(s) for s in dataset.scenario.values],
        }
    )

    # create the dim dict
    ds_dim_dict = {
        "time": time_dim,
        "lat": lat_dim,
        "lon": lon_dim,
        "model": model_dim,
        "scenario": scenario_dim,
    }

    # define the variable(s) and create the var_dict
    ds_var_dict = {}
    for var in dataset.data_vars:
        var_obj = Variable(
            {
                "dimensions": list(dataset[var].dims),
                "type": "data",
                "nodata": str(dataset[var].attrs.get("_FillValue", "nan")),
                "data_type": str(dataset[var].dtype),
                "description": str(dataset[var].attrs.get("long_name", "")),
                "unit": str(dataset[var].attrs.get("units", "")),
            }
        )
        ds_var_dict[var] = var_obj

    # add to dataset info dict
    dataset_info_dict[dataset.attrs["title"]]["dim_dict"] = ds_dim_dict
    dataset_info_dict[dataset.attrs["title"]]["var_dict"] = ds_var_dict

### Create `Collection`, `Item`, and `Asset` from datasets

Create each component from the dataset. This is probably redunant, but again we don't know the search / browse behavior of the STAC catalog, so we will fill the metadata for each component.

In [24]:
for dataset in datasets:

    # grab values from the dataset info dict
    ds_id = dataset_info_dict[dataset.attrs["title"]]["id"]
    ds_desc = dataset_info_dict[dataset.attrs["title"]]["description"]
    ds_bbox = dataset_info_dict[dataset.attrs["title"]]["bbox"]

    # create GeoJSON geometry from the bbox
    ds_geometry = {
        "type": "Polygon",
        "coordinates": [
            [
                [ds_bbox[0], ds_bbox[1]],
                [ds_bbox[0], ds_bbox[3]],
                [ds_bbox[2], ds_bbox[3]],
                [ds_bbox[2], ds_bbox[1]],
                [ds_bbox[0], ds_bbox[1]],
            ]
        ],
    }

    ds_startdate = dataset_info_dict[dataset.attrs["title"]]["startdate"]
    ds_enddate = dataset_info_dict[dataset.attrs["title"]]["enddate"]
    dim_dict = dataset_info_dict[dataset.attrs["title"]]["dim_dict"]
    var_dict = dataset_info_dict[dataset.attrs["title"]]["var_dict"]

    # build the collection
    collection = pystac.Collection(
        id=ds_id,
        description=ds_desc,
        extent=pystac.Extent(
            spatial=pystac.SpatialExtent([ds_bbox]),
            temporal=pystac.TemporalExtent([[ds_startdate, ds_enddate]]),
        ),
        keywords="CMIP6, climate",
        providers=[
            pystac.Provider(
                name="SNAP",
                roles=["producer", "processor"],
                url="https://snap.uaf.edu",
            )
        ],
    )

    # build the item
    item = pystac.Item(
        id=ds_id + "-item",
        geometry=ds_geometry,
        bbox=ds_bbox,
        datetime=None,  # required but we are using start and end datetimes in the datacube extension
        start_datetime=ds_startdate,
        end_datetime=ds_enddate,
        properties={},
    )

    # build the asset
    asset = pystac.Asset(
        href=dataset.encoding["source"],
        media_type=pystac.MediaType.NETCDF,
        roles=["data"],
        title=ds_id + " data",
    )

    # link the asset to the item, the item to the collection, and the collection to the catalog
    item.add_asset("data", asset)
    collection.add_item(item)
    cmip6_catalog.add_child(collection)

    # add the datacube extension to the collection, item, and asset
    for comp in [collection, item, asset]:
        dc = DatacubeExtension.ext(comp, add_if_missing=True)
        dc.dimensions = dim_dict
        dc.variables = var_dict

### Add links and save

In [25]:
dir = "./cmip6_stac"
cmip6_catalog.normalize_hrefs(dir)
cmip6_catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)  # relative paths
# cmip6_catalog.save(catalog_type=pystac.CatalogType.ABSOLUTE_PUBLISHED)  # absolute paths

In [26]:
# use json.dumps to print the entire catalog with all collections, items, and assets
for c in cmip6_catalog.get_children():
    print(c.id)
    print(json.dumps(c.to_dict(), indent=4))
    for i in c.get_items():
        print(i.id)
        print(json.dumps(i.to_dict(), indent=4))
        for a in i.get_assets().values():
            print(a.href)
            print(json.dumps(a.to_dict(), indent=4))

cmip6-monthly-data-on-a-common-grid-with-multi-model-ensemble-mean
{
    "type": "Collection",
    "id": "cmip6-monthly-data-on-a-common-grid-with-multi-model-ensemble-mean",
    "stac_version": "1.1.0",
    "description": "Monthly data from 13 CMIP6 models on a common grid, including multi-model ensemble mean calculated for each variable. Models include CESM2, CNRM-CM6-1-HR, E3SM-2-0, EC-Earth3-Veg, GFDL-ESM4, HadGEM3-GC31-LL, HadGEM3-GC31-MM, KACE-1-0-G, MIROC6, MPI-ESM1-2-HR, MRI-ESM2-0, NorESM2-MM, TaiESM1. Scenarios include historical, ssp126, ssp245, ssp370, ssp585. Variables include clt, evspsbl, hfls, hfss, pr, prsn, psl, rlds, rsds, sfcWind, siconc, snw, tas, tasmax, tasmin, ts, uas, vas. Multi-model ensemble mean is calculated for each variable across all available models for each scenario; some variables do not have data for all models or scenarios.",
    "links": [
        {
            "rel": "root",
            "href": "../catalog.json",
            "type": "application/j

In [27]:
# check out the actual directory structure of the catalog using tree
!tree ./cmip6_stac

./cmip6_stac
├── catalog.json
├── cmip6-climate-indicator-data-on-a-common-grid-with-multi-model-ensemble-mean
│   ├── cmip6-climate-indicator-data-on-a-common-grid-with-multi-model-ensemble-mean-item
│   │   └── cmip6-climate-indicator-data-on-a-common-grid-with-multi-model-ensemble-mean-item.json
│   └── collection.json
└── cmip6-monthly-data-on-a-common-grid-with-multi-model-ensemble-mean
    ├── cmip6-monthly-data-on-a-common-grid-with-multi-model-ensemble-mean-item
    │   └── cmip6-monthly-data-on-a-common-grid-with-multi-model-ensemble-mean-item.json
    └── collection.json

5 directories, 5 files
